In [6]:
import os
from pathlib import Path
import nbformat
from nbclient import NotebookClient
ROOT = Path.cwd()
if ROOT.name == 'model_fits':
    ROOT = ROOT.parent


KERNEL_NAME = os.environ.get("KERNEL_NAME", "python3")

def run_notebook(path: Path, extra_env=None):
    nb = nbformat.read(path, as_version=4)
    client = NotebookClient(nb, timeout=None, kernel_name=KERNEL_NAME, resources={"metadata": {"path": str(ROOT)}})
    if extra_env:
        os.environ.update(extra_env)
    client.execute()
    nbformat.write(nb, path)
    return True


# Run All Notebooks

This notebook runs the full calibration + simulation pipeline in order and logs errors per notebook.


In [7]:
import os
from pathlib import Path
import traceback


In [8]:
try:
    import nbformat
    from nbclient import NotebookClient
except Exception as e:
    raise ImportError("Missing nbformat/nbclient. Install with: python -m pip install nbformat nbclient") from e


In [9]:
# --- Run config (edit if needed) ---
RUN_TAG = os.environ.get("RUN_TAG", "2025Q3")
CALIB_TAG = os.environ.get("CALIB_TAG", RUN_TAG)
PROJ_TAG = os.environ.get("PROJ_TAG", RUN_TAG)
RUN_PROJECTION = os.environ.get("RUN_PROJECTION", "true").lower() in ("1","true","yes","y")  # required to write grade thresholds  # required to write grade thresholds
HIST_END = os.environ.get("HIST_END", "2025Q3")
START_FROM_HIST_END = os.environ.get("START_FROM_HIST_END", "true").lower() in ("1","true","yes","y")
AUTO_PACE_ONEPASS = os.environ.get("AUTO_PACE_ONEPASS", "true").lower() in ("1","true","yes","y")
PACE_CALIB_N_Q = int(os.environ.get("PACE_CALIB_N_Q", "0"))

print("RUN_TAG:", RUN_TAG)
print("CALIB_TAG:", CALIB_TAG)
print("PROJ_TAG:", PROJ_TAG)
print("RUN_PROJECTION:", RUN_PROJECTION)
print("HIST_END:", HIST_END)
print("START_FROM_HIST_END:", START_FROM_HIST_END)
print("AUTO_PACE_ONEPASS:", AUTO_PACE_ONEPASS)
print("PACE_CALIB_N_Q (0=full horizon):", PACE_CALIB_N_Q)

RUN_TWO_PASS = os.environ.get("RUN_TWO_PASS", "false").lower() in ("1","true","yes","y")
print("RUN_TWO_PASS:", RUN_TWO_PASS)


RUN_TAG: 2025Q3
CALIB_TAG: 2025Q3
PROJ_TAG: 2025Q3
RUN_PROJECTION: True
HIST_END: 2025Q3
START_FROM_HIST_END: True
AUTO_PACE_ONEPASS: True
PACE_CALIB_N_Q (0=full horizon): 0
RUN_TWO_PASS: False


In [10]:
from pathlib import Path

# Resolve project root and model_fits directory
cwd = Path.cwd()
if (cwd / 'Distribution_Fits.ipynb').exists():
    mf_dir = cwd
    root = cwd.parent if cwd.name == 'model_fits' else cwd
elif (cwd / 'model_fits' / 'Distribution_Fits.ipynb').exists():
    root = cwd
    mf_dir = cwd / 'model_fits'
else:
    raise FileNotFoundError('Cannot locate model_fits notebooks from cwd')

calib_dir = mf_dir / 'runs' / CALIB_TAG / 'calibration'
proj_dir = mf_dir / 'runs' / PROJ_TAG / 'projection'
calib_dir.mkdir(parents=True, exist_ok=True)
proj_dir.mkdir(parents=True, exist_ok=True)

calib_notebooks = [
    mf_dir / 'Distribution_Fits.ipynb',
    mf_dir / 'Timing_Curves.ipynb',
    mf_dir / 'Omega_Calibration.ipynb',
    mf_dir / 'Copula_Fit.ipynb',
]

all_ok = True
for nb_path in calib_notebooks:
    if not nb_path.exists():
        print('Missing notebook:', nb_path)
        all_ok = False
        continue
    print('Running calibration:', nb_path)
    extra = {
        'RUN_TAG': CALIB_TAG,
        'HIST_END': HIST_END,
        'START_FROM_HIST_END': str(START_FROM_HIST_END),
        'AUTO_PACE_ONEPASS': str(AUTO_PACE_ONEPASS),
        'PACE_CALIB_N_Q': str(PACE_CALIB_N_Q),
    }
    try:
        if not run_notebook(nb_path, extra_env=extra):
            all_ok = False
    except Exception as e:
        print('Failed:', nb_path)
        print(e)
        all_ok = False

# Run simulation with PROJ_TAG, using calibration from CALIB_TAG
if RUN_PROJECTION:
    sim_nb = mf_dir / 'Simulate_Cashflows.ipynb'
    if sim_nb.exists():
        if RUN_TWO_PASS:
            pass1_tag = f"{PROJ_TAG}_pass1"
            print('Running projection pass1:', sim_nb)
            env1 = {
                'RUN_TAG': pass1_tag,
                'HIST_END': HIST_END,
                'START_FROM_HIST_END': str(START_FROM_HIST_END),
                'AUTO_PACE_ONEPASS': str(AUTO_PACE_ONEPASS),
                'PACE_CALIB_N_Q': str(PACE_CALIB_N_Q),
                'FIT_DIR': str(calib_dir),
                'GRADE_THRESH_MODE': 'quarter',
            }
            try:
                run_notebook(sim_nb, extra_env=env1)
            except Exception as e:
                print('Failed:', sim_nb)
                print(e)
                all_ok = False

            sched_path = mf_dir / 'runs' / pass1_tag / 'projection' / 'diagnostics' / 'grade_updates_by_quarter.csv'
            print('Schedule from pass1:', sched_path)

            print('Running projection pass2:', sim_nb)
            env2 = {
                'RUN_TAG': PROJ_TAG,
                'HIST_END': HIST_END,
                'START_FROM_HIST_END': str(START_FROM_HIST_END),
                'AUTO_PACE_ONEPASS': str(AUTO_PACE_ONEPASS),
                'PACE_CALIB_N_Q': str(PACE_CALIB_N_Q),
                'FIT_DIR': str(calib_dir),
                'GRADE_THRESH_MODE': 'quarter',
                'GRADE_SCHEDULE_PATH': str(sched_path),
            }
            try:
                run_notebook(sim_nb, extra_env=env2)
            except Exception as e:
                print('Failed:', sim_nb)
                print(e)
                all_ok = False
        else:
            print('Running projection:', sim_nb)
            envp = {
                'RUN_TAG': PROJ_TAG,
                'HIST_END': HIST_END,
                'START_FROM_HIST_END': str(START_FROM_HIST_END),
                'AUTO_PACE_ONEPASS': str(AUTO_PACE_ONEPASS),
                'PACE_CALIB_N_Q': str(PACE_CALIB_N_Q),
                'FIT_DIR': str(calib_dir),
                'GRADE_THRESH_MODE': 'quarter',
            }
            try:
                run_notebook(sim_nb, extra_env=envp)
            except Exception as e:
                print('Failed:', sim_nb)
                print(e)
                all_ok = False
    else:
        print('Missing notebook:', sim_nb)
        all_ok = False

print('Done. Calibration folder:', calib_dir)
if RUN_PROJECTION:
    print('Projection folder:', proj_dir)

if not all_ok:
    raise SystemExit('Some notebooks failed. See output above.')


Running calibration: /Users/mozeramozali/Desktop/Equity-Cashflow-projection/model_fits/Distribution_Fits.ipynb
Running calibration: /Users/mozeramozali/Desktop/Equity-Cashflow-projection/model_fits/Timing_Curves.ipynb
Running calibration: /Users/mozeramozali/Desktop/Equity-Cashflow-projection/model_fits/Omega_Calibration.ipynb
Running calibration: /Users/mozeramozali/Desktop/Equity-Cashflow-projection/model_fits/Copula_Fit.ipynb
Running projection: /Users/mozeramozali/Desktop/Equity-Cashflow-projection/model_fits/Simulate_Cashflows.ipynb
Done. Calibration folder: /Users/mozeramozali/Desktop/Equity-Cashflow-projection/model_fits/runs/2025Q3/calibration
Projection folder: /Users/mozeramozali/Desktop/Equity-Cashflow-projection/model_fits/runs/2025Q3/projection
